In [2]:
%matplotlib inline

import sys, os

import keras
import pandas as pd
import sklearn as sk
import tensorflow as tf


import numpy as np 
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from tqdm import tqdm

train_directory = os.getcwd() + '/all/train'
test_directory = os.getcwd() + '/all/test'

print train_directory
print test_directory


Using TensorFlow backend.


/Users/wajih_siddiqui/DeepLearning/all/train
/Users/wajih_siddiqui/DeepLearning/all/test


# Clean Training Data

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)
train_generator = datagen.flow_from_directory(
        train_directory,  
        batch_size=1,
        class_mode='categorical')


x, y = train_generator.next()
plt.imshow((x[0]*255).astype('uint8'));
print(list(train_generator.class_indices.keys())[np.argmax(y)])


total_images = 10 # 2750


X_data, Y_data = [], []
for _ in tqdm(range(total_images)):
    x, y = train_generator.next()
    X_data.append(x[0])
    Y_data.append(y[0])
X_data = np.asarray(X_data)
Y_data = np.asarray(Y_data)

# Creating Model

In [ ]:
import numpy as np 
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline


def get_model():
    input_img = Input((256, 256, 3))
    X = BatchNormalization()(input_img)
    X = Convolution2D(16, (3, 3), activation='relu')(X)
    X = BatchNormalization()(X)
    X = Convolution2D(16, (3, 3), activation='relu')(X)
    X = MaxPooling2D()(X)
    X = Convolution2D(32, (3, 3), activation='relu')(X)
    X = BatchNormalization()(X)
    X = Convolution2D(32, (3, 3), activation='relu')(X)
    X = GlobalMaxPooling2D()(X)
#     X = Flatten()(X)
    X = BatchNormalization()(X)
    X = Dense(512, activation='relu')(X)
    X = Dropout(0.2)(X)
    X = Dense(10, activation='softmax')(X)
    model = Model(inputs=input_img, outputs=X)

    model.compile(optimizer='adam', loss='categorical_crossentropy', 
                  metrics=['acc'])
    model.summary()
    return model


model = get_model()

# Fit Model On Data

In [ ]:
model_history = model.fit(X_data, Y_data, batch_size=10, epochs=6, validation_split=0.2,
                          callbacks=[EarlyStopping(monitor='val_acc', patience=3, verbose=1)])

# Testing on Test Data

In [ ]:
# load test images
X_test = []

sample_file_path = os.getcwd() + '/all/' + 'sample_submission.csv'

sub = pd.read_csv(sample_file_path)

for fname in tqdm(sub['fname']):
    filepath = test_directory + '/' + fname
    X_test.append(img_to_array(load_img(filepath, target_size=(256, 256))))

X_test = np.asarray(X_test)

# Making Predictions

In [ ]:
preds = model.predict(X_test, verbose=1)
preds = np.argmax(preds, axis=1)
preds = [list(train_generator.class_indices.keys())[p] for p in tqdm(preds)]


# Using Spark

In [ ]:
# Now that we have the model
# print model


def save_model():
    # serialize model to JSON
    
    if os.path.isfile('model.json'):
        return
    
    
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to disk")
    
def load_model():
    # load json and create model
    json_file = open('model.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("model.h5")
    print("Loaded model from disk")
    return loaded_model


save_model()
loaded_model = load_model()

# Initialize Spark

In [ ]:
import os
import pickle


import sys, os
from skimage.io import imsave, imread

import keras
import pandas as pd
import sklearn as sk
import tensorflow as tf


import numpy as np 
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.callbacks import EarlyStopping
from keras.utils import plot_model
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from pyspark.ml.image import ImageSchema

envs = {'PYSPARK_SUBMIT_ARGS': '"--name" "PySparkShell" "pyspark-shell"', 'SPARK_HOME': '/usr/local/Cellar/apache-spark/2.3.2/libexec', 'LC_CTYPE': 'UTF-8', 'TERM_PROGRAM_VERSION': '400', 'LOGNAME': 'wajih_siddiqui', 'USER': 'wajih_siddiqui', 'NVM_DIR': '/Users/wajih_siddiqui/.nvm', 'HOME': '/Users/wajih_siddiqui', 'PATH': '/Users/wajih_siddiqui/.nvm/versions/node/v6.14.4/bin:/usr/local/opt/python/libexec/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin', 'VIRTUALENVWRAPPER_SCRIPT': '/usr/local/bin/virtualenvwrapper.sh', 'TERM_PROGRAM': 'Apple_Terminal', 'TERM': 'xterm-256color', 'SHELL': '/bin/bash', 'SHLVL': '1', 'OLD_PYTHONSTARTUP': '', 'XPC_FLAGS': '0x0', 'NVM_BIN': '/Users/wajih_siddiqui/.nvm/versions/node/v6.14.4/bin', 'PIP_REQUIRE_VIRTUALENV': 'true', 'PYSPARK_DRIVER_PYTHON': 'python', 'TMPDIR': '/var/folders/cp/n7z5yx_n3d14vv6bd04989g1ktmzwq/T/', 'NVM_CD_FLAGS': '', 'PYTHONHASHSEED': '0', 'JAVA_HOME': '/Library/Java/JavaVirtualMachines/jdk1.8.0_181.jdk/Contents/Home', 'WORKON_HOME': '/Users/wajih_siddiqui/.virtualenvs', 'TERM_SESSION_ID': '78669E14-3E83-48EB-8AC9-EA0591BC0089', 'XPC_SERVICE_NAME': '0', 'PYTHONPATH': '/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/py4j-0.10.7-src.zip:/usr/local/Cellar/apache-spark/2.3.2/libexec/python/:/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/py4j-0.10.7-src.zip:/usr/local/Cellar/apache-spark/2.3.2/libexec/python:/usr/local/Cellar/apache-spark/2.3.2/libexec/python/build:', 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.StzhVlnYIS/Listeners', 'SPARK_SCALA_VERSION': '2.12', 'Apple_PubSub_Socket_Render': '/private/tmp/com.apple.launchd.AUEpexkCG8/Render', 'SPARK_CONF_DIR': '/usr/local/Cellar/apache-spark/2.3.2/libexec/conf', 'VIRTUALENVWRAPPER_PROJECT_FILENAME': '.project', '_SPARK_CMD_USAGE': 'Usage: ./bin/pyspark [options]', 'VIRTUALENVWRAPPER_HOOK_DIR': '/Users/wajih_siddiqui/.virtualenvs', 'PYTHONSTARTUP': '/usr/local/Cellar/apache-spark/2.3.2/libexec/python/pyspark/shell.py', '__CF_USER_TEXT_ENCODING': '0x65AA7F97:0x0:0x0', 'PWD': '/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib', 'SPARK_ENV_LOADED': '1', 'PYSPARK_PYTHON': 'python', 'VIRTUALENVWRAPPER_WORKON_CD': '1'}


for k in envs:
#     if not os.environ.get(k):
    os.environ[k] = envs[k]

# from pyspark import SparkContext
# sc = SparkContext()

# model_data_bc = sc.broadcast(loaded_model.to_json())

# Writing Code to Read Distributed Images

In [1]:
%%time 

import os
from pyspark.ml.image import ImageSchema

def get_images_from_dir(dir_name):
    data_rows = ImageSchema.readImages(dir_name)
    print data_rows.count()
    
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in xrange(0, len(l), n):
        l[i:i + n]


train_directory = os.getcwd() + '/all/train'
test_directory = os.getcwd() + '/all/test'

temp_dir = os.getcwd() + '/all/train/HTC-1-M7/'

images_processed_so_far = 0

image_data_per_camera = {}
            
for camera in os.listdir(train_directory):
    current_directory = '{}/{}/'.format(train_directory, camera)
    
    image_data_per_camera[camera] = []
    
    print "Now Reading images from %s ....." % current_directory
    
    data_rows = ImageSchema.readImages(current_directory)

    print "Processing %s rows" % data_rows.count()
    continue

    for row in data_rows.collect():
        
        print "Going to sleep for 3 secs...."
        time.sleep(3)

        print "Alive...!"
        
        # Read as numpy array
        image_data = ImageSchema.toNDArray(row.image)

        if not image_data_per_camera[camera]:
            image_data_per_camera[camera] = image_data

        else:
            np.concat(image_data_per_camera[camera], image_data)

    images_processed_so_far += len(chunk)

# exit()

# # temp = sc.textFile("image_paths.txt")
# # exit()



# exit()
# all_data = []



# for file_data in temp.collect():
#     img_data = imread(file_data)
#     all_data.append(img_data)
    

# print len(all_data)

    

Now Reading images from /Users/wajih_siddiqui/DeepLearning/all/train/Motorola-Droid-Maxx/ .....


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1164, in send_command
    "Error while receiving", e, proto.ERROR_ON_RECEIVE)
Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:61814)
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
  File "/usr/local/Cellar/python@2/2.7.15_1/Frameworks/Python.framework/Versions/2.7/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 61] Conn

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:61814)
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
  File "/usr/local/Cellar/python@2/2.7.15_1/Frameworks/Python.framework/Versions/2.7/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 61] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:61814)
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
  File "/usr/local/Cellar/python@2/2.7.15_1/Frameworks/Python.framework/Versions/2.7/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:61814)
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
  File "/usr/local/Cellar/python@2/2.7.15_1/Frameworks/Python.framework/Versions/2.7/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 61] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:61814)
Traceback (most recent call last):
  File "/usr/local/Cellar/apache-spark/2.3.2/libexec/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
  File "/usr/local/Cellar/python@2/2.7.15_1/Frameworks/Python.framework/Versions/2.7/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:61814)

In [ ]:
%%time 

import multiprocessing
import scipy
import scipy.misc
import scipy.ndimage

import imageio


image_paths = []


for camera in os.listdir(train_directory):
    current_directory = '{}/{}/'.format(train_directory, camera) 
    
    for x in os.listdir(current_directory):
        image_paths.append(current_directory + x)

    
class Processor:
    def __init__(self,threshold):
        self._threshold=threshold
        self.count = 0

    def __call__(self, filename):
        im = imageio.imread(filename)
        self.count += 1
        
        if not self.count % 100:
            print "Processed %s images so far..." % self.count
            
        return im
    
    
def start():
    proc = Processor(128)
    pool = multiprocessing.Pool()
    
    print "Will process %s images" % len(image_paths)

    results = pool.map(proc, image_paths)
        
    
start()

Will process 2752 images
